# 0. Load API Key

In [1]:
# Load environment
import os
from dotenv import load_dotenv
load_dotenv('../.env', override=True)

True

# 1. Load Document

In [1]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("../Document/HSC26-Bangla1st-Paper.pdf")
doc = loader.load()
len(doc)

49

In [2]:
doc

[Document(metadata={'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'creationdate': '2024-05-13T16:30:00+06:00', 'title': 'PowerPoint Presentation', 'author': 'tariq@10minuteschool.com', 'moddate': '2024-05-13T16:30:00+06:00', 'source': '../Document/HSC26-Bangla1st-Paper.pdf', 'total_pages': 49, 'page': 0, 'page_label': '1'}, page_content='অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা ,\nঅপরিরিতা\nআল ািয রিষয়\nিাাং া\n১ম পত্র'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'creationdate': '2024-05-13T16:30:00+06:00', 'title': 'PowerPoint Presentation', 'author': 'tariq@10minuteschool.com', 'moddate': '2024-05-13T16:30:00+06:00', 'source': '../Document/HSC26-Bangla1st-Paper.pdf', 'total_pages': 49, 'page': 1, 'page_label': '2'}, page_content="১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন?\nক) ডাক্তার্ি খ) ওকালর্ত গ) মাস্টার্ি ঘ) ব্যব্সা\n২। োোলক

In [3]:
# Function to print doc content
def print_doc_content(doc):
    for i, page in enumerate(doc):
        print(f"Page {i + 1}:")
        print(page.page_content)
        print("\n")

print_doc_content(doc)

Page 1:
অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা ,
অপরিরিতা
আল ািয রিষয়
িাাং া
১ম পত্র


Page 2:
১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন?
ক) ডাক্তার্ি খ) ওকালর্ত গ) মাস্টার্ি ঘ) ব্যব্সা
২। োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ, তাি-
ক) প্রর্তপজি খ) প্রভাব্  গ) র্ব্চক্ষণতা ঘ) কূট ব্ুর্ি
র্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও।
র্পতৃহীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতিা। দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও াি ক্ষমতা র্িল না। চাচা 
তাি র্ব্ক ি উকদযাগ র্নকলও যেৌতুক র্নক  ব্া়োব্ার়্ে কিাি কািকণ কনযাি র্পতা অপমার্নত যব্াধ ককি র্ব্ক ি 
আকলাচনা যভকে যদন। দীপু যমক টিি ির্ব্ যদকখ মুগ্ধ হকলও তাি চাচাকক র্কিুই ব্লকত পাকিনর্ন।
৩। েীপুি িািাি সলে ‘অপরিরিতা' গ্লেি দকান িরিলেি রে  আলে?
ক) হর্িকিি খ) মামাি গ) র্িক্ষককি ঘ) র্ব্নুি
৪। উক্ত িরিলে প্রাধানয দপলয়লে -
i) যদৌিাত্ম ii) হীনম্মনযতা   iii) যলাভ
র্নকচি যকানটি ঠিক?
ক। i ও ii  খ। ii ও iii  গ। i ও iii  ঘ। i, ii ও iii
৫. অনুপলেি িয়স কত িেি?
ক) পঁর্চি  খ) িাব্বিি  গ) সাতাি  ঘ) আটাি
প্রাক-মূলযা ন
কতগুকলা প্রকেি সঠিক উিি র্দকত পািকল?


# Indexing / Splitting

In [4]:
# Splitting the document into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)
len(splits)

117

In [5]:
# Index
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
)

retriever = vectorstore.as_retriever()

# Multi-query QUery Transformation

In [7]:
# Prompt

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI


# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate exactly five (5) 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


generate_queries = (
    prompt_perspectives
    | ChatGoogleGenerativeAI(model="gemini-2.5-pro")
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)


In [8]:
# Filter documents retrieval
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)


C:\Users\USER\AppData\Local\Temp\ipykernel_24264\3894762354.py:11: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


20

In [9]:
docs[:4]

[Document(metadata={'total_pages': 49, 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'page_label': '1', 'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'moddate': '2024-05-13T16:30:00+06:00', 'page': 0, 'creationdate': '2024-05-13T16:30:00+06:00', 'author': 'tariq@10minuteschool.com', 'source': '../Document/HSC26-Bangla1st-Paper.pdf', 'title': 'PowerPoint Presentation'}, page_content='অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা ,\nঅপরিরিতা\nআল ািয রিষয়\nিাাং া\n১ম পত্র'),
 Document(metadata={'total_pages': 49, 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'title': 'PowerPoint Presentation', 'source': '../Document/HSC26-Bangla1st-Paper.pdf', 'creationdate': '2024-05-13T16:30:00+06:00', 'author': 'tariq@10minuteschool.com', 'page': 35, 'page_label': '36', 'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'moddate': '2024-05-13T16:30:00+06:00'}, page_content="৫৫। মামািব্ার্হকিিোত্রাপকথিসীমানাকতদূি?\n(ক) আন্দামান পেিন্ত (খ) যকান্নগি পেিন্ত (গ) কানপুি পেিন্ত (

In [11]:
# Generation

from operator import itemgetter

# Final layer of involving LLM
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

"প্রদত্ত তথ্যের উপর ভিত্তি করে, অনুপমের ভাষায় কাকে 'সুপুরুষ' বলা হয়েছে তা উল্লেখ করা নেই।\n\nপ্রদত্ত অংশে শম্ভুনাথ বাবু সম্পর্কে বলা হয়েছে যে, 'তিনি বড়ই চুপচাপ'। কিন্তু 'সুপুরুষ' শব্দটি কোনো চরিত্রের বর্ণনায় ব্যবহার করা হয়নি।"